<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/1992_Schmidhuber_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Yes, there is a direct and significant relation between the 2017 Geoffrey Hinton video on "Fast Weights" and the 1992 Jürgen Schmidhuber paper, "LEARNING TO CONTROL FAST-WEIGHT MEMORIES: AN ALTERNATIVE TO DYNAMIC RECURRENT NETWORKS".

https://www.linkedin.com/feed/update/urn:li:activity:7353235677926285312?utm_source=share&utm_medium=member_desktop&rcm=ACoAAAB0mpcB5qHCHchKrXBczr8mP_dlrFs7rgA

The Schmidhuber paper, published in 1992 (but with the core idea dating back to 1991, as per the document title), introduces the concept of using "fast weights" for temporary memory in neural networks. It describes a system with two feedforward nets: a "slowly learning feed-forward network S" that produces context-dependent weight changes for a "fast-weight network F". The paper explicitly states that "Fast weights can hold on to information over time because they remain essentially invariant unless they are explicitly modified". It also notes that this method allows for "temporary variable binding" through "temporary connectivity patterns".

Geoffrey Hinton's 2017 talk explores the very same core idea: using fast-changing weights in a neural network to store temporary memories. He discusses how this allows for significantly more capacity for temporary storage compared to just neuron activity. Hinton also delves into the historical context, mentioning his own exploration of this idea in 1973 for recursion. Critically, the video highlights that the computational efficiency of fast weights, when sequence length is shorter than the number of hidden units, leads to a mechanism analogous to **attention mechanisms** used in modern machine translation.

Therefore, the 2017 Hinton video can be seen as a re-exploration and a modern interpretation of the "fast weight" concept initially proposed and detailed by Schmidhuber in the early 1990s. While Hinton's talk doesn't directly cite the Schmidhuber 1992 paper in the video itself (based on the summary), the conceptual lineage is clear, particularly with the explicit connection drawn to modern attention mechanisms, which are often cited in the context of the 2017 "Attention Is All You Need" paper. The Schmidhuber paper is even titled "The First Self-Attention in 1991?", directly linking it to the concept of self-attention which gained prominence around the time of Hinton's 2017 lecture.

In [1]:
import numpy as np

class FastWeightNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Slow weights (learnable, persistent)
        # In a real scenario, these would be trained via backpropagation over many examples
        self.W_slow_input_hidden = np.random.rand(input_dim, hidden_dim) * 0.1
        self.W_slow_hidden_output = np.random.rand(hidden_dim, output_dim) * 0.1

        # Fast weights (initially small/zero, will be dynamically modified)
        # We'll represent a single "fast weight" connection for simplicity
        # Let's say it modifies a specific connection from hidden[0] to output[0]
        self.W_fast = 0.0 # This will be changed by a "controller"

        # "Controller" parameters (conceptually, these would be part of a larger S network)
        # For this demo, we'll manually define how an input triggers a fast weight change
        self.fast_weight_controller_rules = {
            "A": 1.0,  # If input is 'A', set fast weight to 1.0
            "B": -1.0, # If input is 'B', set fast weight to -1.0
            "C": 0.0   # If input is 'C', reset fast weight to 0.0
        }

    def forward(self, input_vector, control_signal=None):
        # Step 1: Controller modifies fast weights based on control_signal
        if control_signal in self.fast_weight_controller_rules:
            self.W_fast = self.fast_weight_controller_rules[control_signal]
            print(f"Controller: Input '{control_signal}' changed fast weight to {self.W_fast:.2f}")

        # Step 2: Calculate hidden layer activation using slow weights
        hidden_layer_input = np.dot(input_vector, self.W_slow_input_hidden)
        hidden_layer_output = np.maximum(0, hidden_layer_input) # ReLU activation

        # Step 3: Calculate output layer activation
        # This is where the fast weight comes into play
        # We'll demonstrate it modifying one specific connection
        output_layer_input = np.dot(hidden_layer_output, self.W_slow_hidden_output)

        # Apply the fast weight modification conceptually
        # Let's assume the fast weight *adds* to a specific connection
        # e.g., modifying the influence of hidden_layer_output[0] on output_layer_input[0]
        if self.hidden_dim > 0 and self.output_dim > 0:
            # This is a highly simplified illustration:
            # Imagine W_slow_hidden_output[0, 0] is the base weight.
            # The fast weight dynamically adds to this during a specific forward pass.
            # In a real system, it would be more complex, perhaps
            # modulating a specific connection or a group of connections.
            modified_output_influence = hidden_layer_output[0] * (self.W_slow_hidden_output[0, 0] + self.W_fast)
            output_layer_input[0] += modified_output_influence - (hidden_layer_output[0] * self.W_slow_hidden_output[0, 0])
            # The above line is a simplified way to show the effect:
            # we subtract the original contribution and add the modified one.
            # A more accurate model would have the fast weight directly multiply
            # into the weight matrix used for the current step.


        output_layer_output = 1 / (1 + np.exp(-output_layer_input)) # Sigmoid activation

        return output_layer_output

# --- Demo Usage ---
input_dim = 3
hidden_dim = 5
output_dim = 2

net = FastWeightNetwork(input_dim, hidden_dim, output_dim)

print("--- Initial State ---")
print(f"Initial Fast Weight: {net.W_fast:.2f}\n")

# Scenario 1: Set a temporary memory (e.g., associate input with a modified output behavior)
print("--- Scenario 1: Input 'A' to establish temporary memory ---")
input_vec_A = np.array([1.0, 0.0, 0.0]) # Input for 'A'
output_A_1 = net.forward(input_vec_A, control_signal="A")
print(f"Output after 'A' (fast weight: {net.W_fast:.2f}): {output_A_1}\n")

# Now, process a different input. The fast weight, set by 'A', should persist.
print("--- Scenario 2: Process a neutral input, observe effect of persistent fast weight ---")
input_vec_neutral = np.array([0.5, 0.5, 0.5]) # Neutral input
output_neutral_1 = net.forward(input_vec_neutral) # No control signal
print(f"Output after neutral input (fast weight still {net.W_fast:.2f}): {output_neutral_1}\n")

# Scenario 3: Change the temporary memory (e.g., associate input with a different output behavior)
print("--- Scenario 3: Input 'B' to change temporary memory ---")
input_vec_B = np.array([0.0, 1.0, 0.0]) # Input for 'B'
output_B = net.forward(input_vec_B, control_signal="B")
print(f"Output after 'B' (fast weight: {net.W_fast:.2f}): {output_B}\n")

# Now, process the neutral input again. The fast weight, set by 'B', should persist.
print("--- Scenario 4: Process neutral input again, observe effect of new persistent fast weight ---")
output_neutral_2 = net.forward(input_vec_neutral) # No control signal
print(f"Output after neutral input (fast weight now {net.W_fast:.2f}): {output_neutral_2}\n")

# Scenario 5: Reset the temporary memory
print("--- Scenario 5: Input 'C' to reset temporary memory ---")
input_vec_C = np.array([0.0, 0.0, 1.0]) # Input for 'C'
output_C = net.forward(input_vec_C, control_signal="C")
print(f"Output after 'C' (fast weight: {net.W_fast:.2f}): {output_C}\n")

# Now, process the neutral input again. The fast weight should be reset.
print("--- Scenario 6: Process neutral input again, observe effect of reset fast weight ---")
output_neutral_3 = net.forward(input_vec_neutral) # No control signal
print(f"Output after neutral input (fast weight now {net.W_fast:.2f}): {output_neutral_3}\n")

--- Initial State ---
Initial Fast Weight: 0.00

--- Scenario 1: Input 'A' to establish temporary memory ---
Controller: Input 'A' changed fast weight to 1.00
Output after 'A' (fast weight: 1.00): [0.52772991 0.50514284]

--- Scenario 2: Process a neutral input, observe effect of persistent fast weight ---
Output after neutral input (fast weight still 1.00): [0.52732527 0.50500719]

--- Scenario 3: Input 'B' to change temporary memory ---
Controller: Input 'B' changed fast weight to -1.00
Output after 'B' (fast weight: -1.00): [0.49644863 0.50169117]

--- Scenario 4: Process neutral input again, observe effect of new persistent fast weight ---
Output after neutral input (fast weight now -1.00): [0.48148161 0.50500719]

--- Scenario 5: Input 'C' to reset temporary memory ---
Controller: Input 'C' changed fast weight to 0.00
Output after 'C' (fast weight: 0.00): [0.50267191 0.50318047]

--- Scenario 6: Process neutral input again, observe effect of reset fast weight ---
Output after neut

The provided output demonstrates a conceptual model of a neural network using "fast weights" for temporary memory.

Here's a breakdown of what the output shows:

* **Initial State:** The `Fast Weight` starts at `0.00`, indicating no temporary memory has been established yet.
* **Scenario 1 (Input 'A'):** When input 'A' is processed, a "Controller" (conceptually, a part of the network responsible for modifying fast weights) changes the `Fast Weight` to `1.00`. The output of the network is then calculated based on this new fast weight. This simulates "establishing a temporary memory" or association.
* **Scenario 2 (Neutral Input):** A "neutral" input is given, but crucially, *no* control signal is provided to change the fast weight. The `Fast Weight` remains at `1.00`, and the network's output reflects the influence of this persistent fast weight. This demonstrates that the temporary memory, once set, persists and influences subsequent processing.
* **Scenario 3 (Input 'B'):** Input 'B' acts as a new control signal, prompting the Controller to change the `Fast Weight` to `-1.00`. This signifies a change in the temporary memory or a re-binding of an association. The network's output changes accordingly.
* **Scenario 4 (Neutral Input again):** The neutral input is processed again. The `Fast Weight` is now `-1.00`, and the output differs from Scenario 2, showcasing the effect of the new temporary memory.
* **Scenario 5 (Input 'C'):** Input 'C' causes the Controller to reset the `Fast Weight` back to `0.00`. This simulates clearing or disassociating the temporary memory.
* **Scenario 6 (Neutral Input again):** The neutral input is processed one last time. With the `Fast Weight` at `0.00`, the output is different again, showing that the influence of the temporary memory has been removed.

In essence, this output visually confirms how the "fast weight" mechanism allows for dynamic, short-term changes in network behavior based on preceding "control signals" (which would be learned contexts in a full model), thereby acting as a form of temporary associative memory. The "slow weights" (not explicitly shown changing in this output) would represent the network's long-term, stable knowledge.